# Tuto spisem, co vsetko mam
- Explorativna analyza
- Agresivita samce samice, ale celkom dobre
- Kastracia
- Kastracia pokracovanie
- Kastracia pokracovanie
- Agresivita psov vs vek (este doplnit brechanie treba)
- Impulzivita a Nepozornost vs vek
- Strach vs vek
- Najtazsie a najlahsie plemena (mohol by som skusit spravit nejaku zavislost na agresivite v zavislosti od velkosti plemena)
- Agresivita vs Strach - zaujimave
- Agresivita vs citlivost na hluk
- Strach vs citlivost na hluk

### Napady
- 

# Napady na hypotezy
- v suvislosti oddelenia od matky, bude psom najprv stupat a potom klesat, nebojacnost a veselost (vitalita)
### Najprv zistenia
- Samci su agresivnejsi k vlastnemmu pohlaviu, a samice k opacnemu, ale iba o trochu, samci su agresivnejsi k ludom


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import Markdown
# from google.colab import files
# uploaded = files.upload()

### Prikazy na stiahnutie z excelu a ulozenie do csv kvoli nacitavaniu
# dog_info = pd.read_excel("data-for-publication.xlsx", sheet_name="Information", header=0)
# dog_info.to_csv("psy_info.csv")
# files.download("psy_info.csv")

## Stiahnem data a ulozim ich v csv a vyrobim dataframe dogs a dogs info

In [ ]:
dogs = pd.read_csv("psy_data.csv", header=0).set_index("dog.id")
dog_info = pd.read_csv("psy_info.csv", header=0)
casti = ['Demographic information', 'Environment and background', 'Personality', 
     'Noise sensitivity', 'Fearfulness', 'Aggression', 
     'Fear of surfaces and heights', 'Separation anxiety', 'Impulsivity/ inattention']
zoz = []
for i in dog_info['SECTION']:
    if i in casti:
        slovo = i
    zoz.append(slovo)
dog_info['SECTION'] = pd.Series(zoz)
dog_info = dog_info.set_index('SECTION').drop('Unnamed: 0', axis=1)


## Tuto budu zoznamy jednotlivych casti, a urobene casti tabulky podla kategorie


In [ ]:
spojite_vsetky = dog_info[dog_info['COLUMN VALUES'].str.contains('continuous', na=False)]
spojite = list(spojite_vsetky.query('SECTION != "Demographic information"')['COLUMN NAME'])
spojite_set = set(spojite)

dogs_male = dogs.query("sex=='male'")
dogs_female = dogs.query("sex=='female'")

dogs_demographic = dogs.transpose().iloc[0:11].transpose()
dogs_environment = dogs.transpose().iloc[11:42].transpose()
dogs_personality = dogs.transpose().iloc[42:112].transpose()
dogs_noise_sensitivity = dogs.transpose().iloc[112:154].transpose()
dogs_fearfullness = dogs.transpose().iloc[154:189].transpose()
dogs_aggression = dogs.transpose().iloc[189:214].transpose()
dogs_fear_surfaces = dogs.transpose().iloc[214:222].transpose()
dogs_separation_anxiety = dogs.transpose().iloc[222:232].transpose()
dogs_inattention = dogs.transpose().iloc[232:250].transpose()


demographic = list(dogs_demographic.columns)
environment = list(dogs_environment.columns)
personality = list(dogs_personality.columns)
noise_sensitivity = list(dogs_noise_sensitivity.columns)
fearfullness = list(dogs_fearfullness.columns)
aggression = list(dogs_aggression.columns)
fear_surfaces = list(dogs_fear_surfaces.columns)
separation_anxiety = list(dogs_separation_anxiety.columns)
inattention = list(dogs_inattention.columns)

demographic_continuous = list(spojite_set & set(demographic))
environment_continuous = list(spojite_set & set(environment))
personality_continuous = list(spojite_set & set(personality))
noise_sensitivity_continuous = list(spojite_set & set(noise_sensitivity))
fearfullness_continuous = list(spojite_set & set(fearfullness))
aggression_continuous = list(spojite_set & set(aggression))
fear_surfaces_continuous = list(spojite_set & set(fear_surfaces))
separation_anxiety_continuous = list(spojite_set & set(separation_anxiety))
inattention_continuous = list(spojite_set & set(inattention))

demographic_discrete = list(set(demographic) - spojite_set)
environment_discrete = list(set(environment) - spojite_set)
personality_discrete = list(set(personality) - spojite_set)
noise_sensitivity_discrete = list(set(noise_sensitivity) - spojite_set)
fearfullness_discrete = list(set(fearfullness) - spojite_set)
aggression_discrete = list(set(aggression) - spojite_set)
fear_surfaces_discrete = list(set(fear_surfaces) - spojite_set)
separation_anxiety_discrete = list(set(separation_anxiety) - spojite_set)
inattention_discrete = list(set(inattention) - spojite_set)


# Explorativna analyza

In [ ]:
breeds = list(dogs['breed_group'].unique())
breed_count = len(breeds)
male_count = len(dogs.query("sex=='male'"))
female_count = len(dogs.query("sex=='female'"))

# print('plemena:', breeds)
print('mnozstvo samcov:', male_count)
print('mnozstvo samic:', female_count)
print('nozstvo plemien:', breed_count)


figure, axes = plt.subplots(1, 1, figsize=(13, 10))
sns.violinplot(data=dogs[spojite], orient='h', ax=axes)

figure, axes1 = plt.subplots(1, 1, figsize=(13, 10))
sns.violinplot(data=dogs[aggression], orient='h', ax=axes1)


## Agresivita
#### samce vs samice

In [ ]:

figure, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
agresivita_gender = dogs.groupby('sex')[['Aggressive_dogs_samegender', 'Aggressive_dogs_oppositegender', 'Aggressive_people']].mean().reset_index().melt(id_vars='sex')
agresivita_gender_median = dogs.groupby('sex')[['Aggressive_dogs_samegender', 'Aggressive_dogs_oppositegender', 'Aggressive_people']].median().reset_index().melt(id_vars='sex')
# display(agresivita_gender)
sns.barplot(agresivita_gender, y='variable', x='value', hue='sex', palette={'male':'blue', 'female':'violet'}, ax=axes[0], orient='h')

figure, axes = plt.subplots(1, 1)
plt.xticks(ha='right') 
agresivita_gender = dogs.groupby('sex')[['aggressiveness_dominance_score', 'stranger_aggression_score', 'owner_aggression_score', 'dog_aggression_score', 'barking_score']].mean().reset_index().melt(id_vars='sex')
# display(agresivita_gender)
sns.barplot(agresivita_gender, x='variable', y='value', hue='sex', palette={'male':'blue', 'female':'violet'})
axes.tick_params(axis='x', rotation=25)




In [ ]:

figure, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
agresivita_gender = dogs.groupby('sex')[['Aggressive_dogs_samegender', 'Aggressive_dogs_oppositegender', 'Aggressive_people']].median().reset_index().melt(id_vars='sex')
# display(agresivita_gender)
sns.barplot(agresivita_gender, y='variable', x='value', hue='sex', palette={'male':'blue', 'female':'violet'}, ax=axes[0], orient='h')
# sns.violinplot(data=dogs[['Aggressive_dogs_samegender', 'Aggressive_dogs_oppositegender', 'Aggressive_people']], ax=axes[1], orient='h')


figure, axes = plt.subplots(1, 1)
agresivita_gender = dogs.groupby('sex')[['aggressiveness_dominance_score', 'stranger_aggression_score', 'owner_aggression_score', 'dog_aggression_score', 'barking_score']].mean().reset_index().melt(id_vars='sex')
# display(agresivita_gender)
sns.barplot(agresivita_gender, x='variable', y='value', hue='sex', palette={'male':'blue', 'female':'violet'})
axes.tick_params(axis='x', rotation=25)




# Kastracia
zistovanie najrozdielnejsich vlastnosti v ramci kastracie

In [ ]:
pocet = 10
pripocet = 1

kastracia_male = dogs.query('sex=="male"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
kastracia_male.columns.name = None
kastracia_male["personality_trait"] = kastracia_male.index

kastracia_rozdiely_male = kastracia_male.copy(deep=True)
kastracia_rozdiely_male["diff"] = abs(kastracia_rozdiely_male["intact"] - kastracia_rozdiely_male["sterilized"])
kastracia_rozdiely_male["relative_diff"] = abs((kastracia_rozdiely_male["intact"] - kastracia_rozdiely_male["sterilized"])/(abs(kastracia_rozdiely_male["intact"])+abs(kastracia_rozdiely_male["sterilized"])+pripocet))
# kastracia_rozdiely_male["relative_diff"] = abs((kastracia_rozdiely_male["intact"] - kastracia_rozdiely_male["sterilized"])/(kastracia_rozdiely_male["intact"]))
# toto bol povodny vypocet relativnej zmeny, ale zmenil som o trochu na ten nadnim

index_male_diff = list(kastracia_rozdiely_male.sort_values('diff', ascending=False).head(pocet).index)
index_male_rel_diff = list(kastracia_rozdiely_male.sort_values('relative_diff', ascending=False).head(pocet).index)

###############################################################################################################################

kastracia_female = dogs.query('sex=="female"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
kastracia_female.columns.name = None
kastracia_female["personality_trait"] = kastracia_female.index

kastracia_rozdiely_female = kastracia_female.copy(deep=True)
kastracia_rozdiely_female["diff"] = abs(kastracia_rozdiely_female["intact"] - kastracia_rozdiely_female["sterilized"])
kastracia_rozdiely_female["relative_diff"] = abs((kastracia_rozdiely_female["intact"] - kastracia_rozdiely_female["sterilized"])/(abs(kastracia_rozdiely_female["intact"])+abs(kastracia_rozdiely_female["sterilized"])+pripocet))
# kastracia_rozdiely_female["relative_diff"] = abs((kastracia_rozdiely_female["intact"] - kastracia_rozdiely_female["sterilized"])/kastracia_rozdiely_female["intact"])
# zmenil som vypocet na odkomentovany

index_female_diff = list(kastracia_rozdiely_female.sort_values('diff', ascending=False).head(pocet).index)
index_female_rel_diff = list(kastracia_rozdiely_female.sort_values('relative_diff', ascending=False).head(pocet).index)


## Kastracia pokracovanie - Grafy
vygrafovanie zistenych vlastnosti

In [ ]:
dogs_melted_male_diff = kastracia_male.loc[index_male_diff, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_female_diff = kastracia_female.loc[index_female_diff, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(21, 7), sharey=True)
sns.barplot(data=dogs_melted_male_diff, x=dogs_melted_male_diff.index, y="value", hue="sterilization_status", ax=axes[0])
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_female_diff, x=dogs_melted_female_diff.index, y="value", hue="sterilization_status", ax=axes[1])
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

###########################################

dogs_melted_male_rel_diff = kastracia_male.loc[index_male_rel_diff, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_female_rel_diff = kastracia_female.loc[index_female_rel_diff, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(21, 7), sharey=True)
sns.barplot(data=dogs_melted_male_rel_diff, x=dogs_melted_male_rel_diff.index, y="value", hue="sterilization_status", ax=axes[0])
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_female_rel_diff, x=dogs_melted_female_rel_diff.index, y="value", hue="sterilization_status", ax=axes[1])
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

## Kastracia dalsia
planujem urobit ze zoberiem spojite premenne z jednotlivych casti dotaznika
a porovnam s ktorou suvisi kastrovanie najviac (nemam nicim podlozene, ale pride mi ze pri jednotlivych otazkach to moze byt aj trochu nahodne)

In [ ]:
dogs_melted_spojite_male = kastracia_male.loc[spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_spojite_female = kastracia_female.loc[spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")


figure, axes = plt.subplots(2, 1, figsize=(14, 15))
sns.barplot(data=dogs_melted_spojite_male, x=dogs_melted_spojite_male.index, y="value", hue="sterilization_status", ax=axes[0])
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_spojite_female, x=dogs_melted_spojite_female.index, y="value", hue="sterilization_status", ax=axes[1])
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

## Kastracia a agresivita
porovnam podobnym bargrafom agresivitu

In [ ]:
dogs_melted_aggression_male = kastracia_male.loc[aggression_discrete, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_aggression_female = kastracia_female.loc[aggression_discrete, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(9, 8), sharey=True)
sns.barplot(data=dogs_melted_aggression_male, y=dogs_melted_aggression_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h')
# axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_aggression_female, y=dogs_melted_aggression_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h')
# axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")


## Kastracia dalsia


In [ ]:
dogs_melted_personality_male = kastracia_male.loc[personality_continuous, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_personality_female = kastracia_female.loc[personality_continuous, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=True)
sns.barplot(data=dogs_melted_personality_male, y=dogs_melted_personality_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h')
# axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_personality_female, y=dogs_melted_personality_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h')
# axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

figure, axes1 = plt.subplots(1, 2, figsize=(9, 4), sharey=True)

sns.violinplot(dogs.query('sex=="male"')[personality_continuous], ax=axes1[0], orient='h')
sns.violinplot(dogs.query('sex=="female"')[personality_continuous], ax=axes1[1], orient='h')


### Tuto je agresivita psov
##### ku psom, majitelom a cudzncom
podla veku 

In [ ]:
psy1 = dogs.copy(deep=True)
psy1["Aggression_age"] = psy1["Aggression_age"].round(1)
psy1 = psy1.groupby("Aggression_age")[aggression_continuous].mean()

ax = sns.histplot(data=dogs, x="Aggression_age")


figure, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)
sns.scatterplot(x=psy1.index, y=psy1["stranger_aggression_score"], ax=axes[0])
axes[0].set_title("Stranger Aggression Age")
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Aggression Score")

# Scatterplot 2
sns.scatterplot(x=psy1.index, y=psy1["owner_aggression_score"], ax=axes[1])
axes[1].set_title("Owner Aggression and Age")
axes[1].set_xlabel("Age")

# Scatterplot 3
sns.scatterplot(x=psy1.index, y=psy1["dog_aggression_score"], ax=axes[2])
axes[2].set_title("Dog Aggression and Age")
axes[2].set_xlabel("Age")

plt.tight_layout()
plt.show()


### Teraz spravim podobne impulzivitu a nepozornost
nasiel som jemnu zavislost, impulzivita stupa a vrcholi okolo 1 roka, potom pomaly klesa, nepozornost je na tom velmi podobne



In [ ]:
psy2 = dogs.copy(deep=True)
psy2["Impulsivity_age"] = psy2["Impulsivity_age"].round(1)
psy2 = psy2.groupby("Impulsivity_age")[inattention_continuous].mean()


ax = sns.histplot(data=dogs, x="Impulsivity_age")

figure, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)
sns.scatterplot(x=psy2.index, y=psy2["inattention_score"], ax=axes[0])
axes[0].set_title("Inattention vs Age")
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Inattention score")

# Scatterplot 2
sns.scatterplot(x=psy2.index, y=psy2["impulsivity_score"], ax=axes[1])
axes[1].set_title("Impulsivity vs Age")
axes[1].set_xlabel("Age")
axes[1].set_ylabel("Impulsivity Score")

plt.tight_layout()
plt.show()

### Teraz spravim podobne strach
pri strachu ale nevidim ziadnu suvislost




In [ ]:
psy3 = dogs.copy(deep=True)
psy3["Fearfulness_age"] = psy3["Fearfulness_age"].round(1)
f = psy3.groupby("Fearfulness_age")["fearfulness_score"].mean()
f = pd.DataFrame(f)
ax = sns.histplot(data=psy3, x="Fearfulness_age")

figure, axes = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.scatterplot(x=f.index, y=f["fearfulness_score"], ax=axes)

plt.tight_layout()
plt.show()



### Priemerne hmotnosti jednotlivych plemien barplot


In [ ]:

psy_hmostnost_max = pd.DataFrame(dogs.groupby('breed_group')['weight_kg'].mean().sort_values()).head()
psy_hmostnost_min = pd.DataFrame(dogs.groupby('breed_group')['weight_kg'].mean().sort_values()).tail()

figure, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
sns.barplot(data=psy_hmostnost_max, x=psy_hmostnost_max.index, y='weight_kg', ax=axes[0])
axes[0].tick_params(axis='x', rotation=45)
sns.barplot(data=psy_hmostnost_min, x=psy_hmostnost_min.index, y='weight_kg', ax=axes[1])
axes[1].tick_params(axis='x', rotation=45)


pocty_psov = pd.DataFrame(dogs.groupby("breed_group").count().sort_values(by="sex"))
# figure, axes1 = plt.subplots(1, 1)
# display(pocty_psov)
# sns.histplot(data=pocty_psov, ax=axes1)

### Teraz Spravim Agresivitu vs Strach

In [ ]:

psy4 = dogs.copy(deep=True)
psy4["fearfulness_score"] = psy4["fearfulness_score"].round(1)

psy4 = psy4.groupby("fearfulness_score")[aggression_continuous].mean()

figure, ax = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
ax = sns.histplot(data=dogs, x="fearfulness_score")

figure, axes = plt.subplots(2, 2, figsize=(13, 9), sharey=True)
sns.scatterplot(x=psy4.index, y=psy4["stranger_aggression_score"], ax=axes[0][0])
axes[0][0].set_title("Stranger Agression Fearfulness")

sns.scatterplot(x=psy4.index, y=psy4["owner_aggression_score"], ax=axes[0][1])
axes[0][1].set_title("Owner Agression Fearfulness")

sns.scatterplot(x=psy4.index, y=psy4["dog_aggression_score"], ax=axes[1][0])
axes[1][0].set_title("Dog Agression Fearfulness")

sns.scatterplot(x=psy4.index, y=psy4["barking_score"], ax=axes[1][1])
axes[1][1].set_title("Barking Fearfulness")



### Citlivost na Hluk vs Agresivita

In [ ]:
psy5 = dogs.copy(deep=True)
psy5["noise_sensitivity_score"] = psy5["noise_sensitivity_score"].round(1)

psy5 = psy5.groupby("noise_sensitivity_score")[aggression_continuous].mean()
psy5 = pd.DataFrame(psy5)

figure, ax = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
ax = sns.histplot(data=dogs, x="noise_sensitivity_score")

figure, axes = plt.subplots(2, 2, figsize=(13, 9), sharey=True)
sns.scatterplot(x=psy5.index, y=psy5["stranger_aggression_score"], ax=axes[0][0])
axes[0][0].set_title("Stranger Agression vs noise sensitivity")

sns.scatterplot(x=psy5.index, y=psy5["owner_aggression_score"], ax=axes[0][1])
axes[0][1].set_title("Owner Agression vs noise sensitivity")

sns.scatterplot(x=psy5.index, y=psy5["dog_aggression_score"], ax=axes[1][0])
axes[1][0].set_title("Dog Agression vs noise sensitivity")

sns.scatterplot(x=psy5.index, y=psy5["barking_score"], ax=axes[1][1])
axes[1][1].set_title("Barking vs noise sensitivity")

### Teraz strach vs citlivost na hluk

In [ ]:
psy6 = dogs.copy(deep=True)
psy6["noise_sensitivity_score"] = psy6["noise_sensitivity_score"].round(1)

psy6 = psy6.groupby("noise_sensitivity_score")[["fearfulness_score"]].mean()

figure, ax = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
ax = sns.histplot(data=dogs, x="noise_sensitivity_score")

figure, axes = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.scatterplot(x=psy6.index, y=psy6["fearfulness_score"], ax=axes)
axes.set_title("Fearfulness vs noise sensitivity")


### Este dokoncit tretiu agresivitu

## Fearfullnes vs noise sensitivity
#### samci vs samice

In [ ]:
sns.barplot()

# Environment

In [ ]:
# ['ownership_reason', 'obtained_from', 'age_enter_household', 'weaning_age', 'Socialization_men', 
#  'Socialization_women', 'Socialization_children', 'Socialization_dogs', 'Socialization_city', 
#  'Socialization_car', 'Socialization_public.transportation', 'socialization', 'socialization_quantile', 
#  'living_place', 'living_building', 'outdoor_access', 'other_dogs_when_obtained', 'currently_other_dogs', 
#  'daily_exercise', 'leashed_unleashed', 'trains_agility', 'trains_obedience_rally', 'trains_nose_work', 
#  'trains_rescue_utility', 'hobby_frequency_home', 'hobby_frequency_out', 'alone_time', 'life_experience', 
#  'dog_experience', 'dog_breeder', 'animal-related_work']

# ['Willing_to_learn', 'Playful_dogs', 'Predictable', 'Obedient', 'Boisterous', 'Sensitive_touch', 
#  'Vocal', 'Anxious', 'Independent', 'Lazy', 'Intelligent', 'Reliable', 'Prey_driven_chase', 
#  'Human_dependent', 'Wary', 'Erratic', 'Stubborn', 'Fearful_people', 'Playful_people', 'Playful_alone', 
#  'Curious', 'Fearful_dogs', 'Impulsive', 'Affectionate_people', 'Decisive', 'Persevering', 'Distractible', 
#  'Submissive', 'Cautious', 'Restless', 'Confident', 'Easygoing', 'Dominant', 'Patient', 'Sociable_dogs_samegender', 
#  'Slow', 'Sociable_dogs_oppositegender', 'Calm', 'Provocative', 'Excitable', 'Human_oriented', 'Focused', 'Prey_driven_search', 
#  'Empathic', 'Aggressive_dogs_samegender', 'Greedy', 'Attention_seeking', 'Indifferent', 'Aggressive_people', 
#  'Affectionate_dogs', 'Vigilant', 'Calming', 'Solitary', 'Aggressive_dogs_oppositegender', 'Territorial', 'Flexible', 
#  'Easily_recovered', 'Bold', 'Active', 'Energetic', 'Insecure', 'Attentive', 'Sociable_people', 'insecurity_score', 
#  'training_focus_score', 'activity_playfulness_score', 'aggressiveness_dominance_score', 
#  'human_sociability_score', 'dog_sociability_score', 'perseverance_score']

order=['under_7_weeks', '7_weeks', '8_weeks', '9_weeks', 'over_9_weeks', 'still_with_dam',
       'unknown']

order=['under_1_hour', '1-2_hours', '2-3_hours', 'over_3_hours']

psy7 = dogs.copy(deep=True)
psy7 = psy7.groupby("daily_exercise").agg('mean', numeric_only=True)[personality]
psy7 = psy7.loc[order, :]
# display(psy7)

figure, ax = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.countplot(data=dogs, x="daily_exercise", ax=ax)
ax.tick_params(axis='x', rotation=25)

o=6
figure, axes = plt.subplots(len(personality)//7, 1, figsize=(5, 33))
for i in range(len(o*personality)//7, (o+1)*len(personality)//7):
       sns.barplot(y=psy7.index, x=psy7[personality[(i)]], ax=axes[i%10])


# daily exercise: 
# curious, fearful_dogs, decisive, perservering, cautious, playful, obedient, anxious, lazy, 
# intelligent, playful_alone, dominant, sociable_dogs_samegender, human_oriented, focused, Affectionate_people, Energetic, 
# +vsetky spojite vyzeraju zaujimavo
# 
# 
# 
# 
# 
# 
# 


